In [ ]:
top_annotation

<table>
  <tr>
    <th>
      <img src="https://raw.githubusercontent.com/riinbre-bioinfo/Colab_Biomarkers/main/Biomarkers/images/RIINBRE-Logo.jpg", height = "125", alt="RI-INBRE Logo">
    </th>
    <th>
      <img src="https://raw.githubusercontent.com/riinbre-bioinfo/Colab_Biomarkers/main/Biomarkers/images/MIC_Logo.png", height = "125", alt="RI-INBRE Logo">
    </th>
  </tr>
</table>

# Analysis of Biomedical Data for Biomarker Discovery
## Submodule 7: Exploratory Analysis of Proteomics IRI Data
### Dr. Christopher L. Hemme
### Director, [RI-INBRE Molecular Informatics Core](https://web.uri.edu/riinbre/mic/)
### The University of Rhode Island College of Pharmacy
Last Updated: June 5, 2023

---

## Introduction

Omics data allows us to evaluate potential biomarkers on a systematic scale.  Instead of picking and choosing individual biomarkers, omics data allows us to identify large numbers of correlated variables that may indicate broad biological trends.  There are several benefits of such a strategy.  First, we may identify unknown or unanticipated biomarkers that were not obvious candidates but which are clearly correlated to (but not necessarily causative of) disease states.  Second, we can map this data to biological networks, allowing us to identify metabolic pathways correlated to disease states or to identify regulatory networks that can indicate mechanisms by which the state changes are occurring.  The cost of using omics data however is the so-called "curse of dimensionality".  Omics data involves thousands of features (e.g. expressed genes, active proteins, etc.), most of which are not actively changed across states.  Identifying the correlated variables and determining their statistical and biological significance is a challenge when dealing with a single omics layer such as the proteome.  The challenge becomes even greater when attempting to correlate results across different omics layers (known as multiomics) or when comparing similar results across laboratories (meta-analysis).  For this module, we will focus on a traditional bulk proteomics experiment from the rat renal IRI data to identify proteins significantly affected by IRI.

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> Blue boxes will indicate helpful tips.</div>

<div class="alert alert-block alert-warning">
<b>&#127891; Note:</b> Used for interesting asides or notes.
</div>

<div class="alert alert-block alert-success">
<b>&#9997; Reference:</b> This box indicates a reference for an attached figure or table.
</div>

<div class="alert alert-block alert-danger">
<b>&#128721; Caution:</b> A red box indicates potential hazards or pitfalls you may encounter.
</div>

---

## Identification of Proteomic Biomarkers

### Pre-Analysis Prep

Let's start by loading the required R packages and our experimental object.

In [ ]:
if(!require(devtools)) install.packages("devtools")
devtools::install_github("kassambara/factoextra")

if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
library('BiocManager')

bioc_packages <- c("limma", "ComplexHeatmap", "M3C")
installed_bioc_packages <- bioc_packages %in% rownames(installed.packages())
if (any(installed_bioc_packages == FALSE)) {BiocManager::install(bioc_packages[!installed_bioc_packages])}

BiocManager::install("preprocessCore")
BiocManager::install("preprocessCore", configure.args="--disable-threading", force = TRUE)

packages <- c("plyr", "tidyverse", "factoextra")
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {install.packages(packages[!installed_packages])}

In [ ]:
require('preprocessCore')
require('limma')
require('ComplexHeatmap')
require('plyr')
require('tidyverse')
require('factoextra')

Remember to create a "data" folder in Colab and upload your previous rds file from champter 6.

In [ ]:
# You will need to create a data directory in Colab.  Click on the folder in the left-hand frame, right click, choose "New Folder", and name it 'data'.  You can then upload exp_obj.rds to the 'data' folder.
exp_obj <- readRDS(file = "data/exp_obj.rds")

### Data Normalization

When working with data, it's often a good idea to work with copies of your data instead of the raw data itself to avoid accidentally modifying the original data.  In our case, it's not a big deal since we could just reload the experimental object, but we can save ourselves some typing by saving the proteome data and metadata into separate variables.

In [ ]:
#loading the experimental object and extract the metadata
proteome <- exp_obj$data$proteomics$log2
metadata <- exp_obj$metadata
metadata

Remember that our data was log2 transformed.  Because the proteomic signal data spans a wide range of values, log transforming the data linearizes the data making it more normally distributed.  Let's look at a boxplot of the data to see how it looks.

In [ ]:
head(proteome)

In [ ]:
boxplot(proteome)

We can go a step further and quantile normalize the data.  In quantile normalization, we are making the distribution of the data in each sample identical in their statistical properties.  We will use the <i>normalize.quantiles</i> function from the <b>preprocessCore</b> package to do the normalization. This step is important to decrease variations or noise that could cause results to skew within the dataset. This also provides power to downstream analysis by grouping liked values together to select important features which we will be doing in submodule 9. 

<div class="alert alert-block alert-success">
<b>&#9997; Reference:</b> For more information see the https://academic.oup.com/bib/article/19/1/1/2562889
</div>

In [ ]:
#normalize the proteome data
proteome_norm <- normalize.quantiles(proteome)
rownames(proteome_norm) <- rownames(proteome)
colnames(proteome_norm) <- colnames(proteome)
head(proteome_norm)

In [ ]:
boxplot(proteome_norm)

<div class="alert alert-block alert-danger">
<b>&#128721; Caution:</b> There are many ways to normalize data, and even different strategies for using quantile normalization.  No normalization method is perfect and some may even introduce error into your data.  Tools such as the <b>NormalyzerDE</b> package in BioConductor can help you determine the best normalization method for your data.
</div>

### Dimensionality Reduction (PCA and MDS)

Now we'll begin the exploratory analysis.  Let's start with a principal component analysis (PCA).  PCA is useful to get a feel for similarities in our sample proteomes and if there are any batch effects we need to account for.  PCA is a dimensionality reduction technique in which we are linearly transforming our data to a new set of axes (called principal components) that define the variability in the data.  Typically, the first 2-3 principal components (PC) account for the majority of variation in the data, making visualizing the data much easier.  PCA also allows us to identify groups of correlated variables which help separate groups of similar samples (see biplots below). 

There are many ways to calculate and visualize PCA data in R.  We'll use the base R <i>prcomp</i> function and the visualization tools in the <b>factoextra</b> package.  First, let's create a scree plot to see which of our principal components are relevant.  The scree plot is a bar chart showing the amount of variability accounted for by each PC.

In [ ]:
#the Dimensions shown in the plot below can also be called Principal Components
proteome_pca <- prcomp(t(na.omit(proteome_norm)), center = TRUE, scale. = TRUE)
fviz_eig(proteome_pca)

<div class="alert alert-block alert-warning">
    <b>&#127891; Note:</b> Reminder that PCA decomposes the data matrix <b>X</b> into matrices <b>U</b> and <b>V</b>, where

$$X = U \cdot V = UV^T$$

$U$ = Scores matrix, which is the original data rotated into the new coordinate system

$V$ = Loadings matrix, which is the weights applied to each data point in the new coordinate system
</div>

Most of the variability seems to be in the first two PC's, so that's what we'll plot.  Often you will see on a PCA plot that we include the information for both the individual data points and the variables (we call this a biplot).  This is often useful to us because it gives us an indication of which variables are defining the groups we see in our PCA.  We won't plot the variables right now since adding 2000+ proteins to the plot will be too messy to be useful.  In general though, the biplot tends to follow the same patterns as the linear regression results from differential analysis, which we will see in the next chapter.

In [ ]:
#plot the PCA plot by Treatment
fviz_pca_ind(proteome_pca,
             col.ind = metadata$Treatment,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Treatment",
             pointsize = 3,
             label = "none"
             )
#plot the PCA plot by Time
fviz_pca_ind(proteome_pca,
             col.ind = metadata$Time,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Time",
             pointsize = 3,
             label = "none"
             )

At first glance, there appears to be a strong separation by time, with earlier time points primarily in their own cluster.  However, if you remember, this data was collected in batches and therefore we have to account for any potential batch effects.  Let's replot the data based on that batch column.

In [ ]:
fviz_pca_ind(proteome_pca,
             col.ind = metadata$Batch,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Batch",
             pointsize = 3,
             label = "none"
             )

We can now clearly see that this separation is based on the batch effect which must be corrected.  For the purposes of exploratory analysis, we will use the <i>removeBatchEffect</i> function in <b>limma</b>.

<div class="alert alert-block alert-danger">
<b>&#128721; Caution:</b> Batch correction can be tricky in that it can sometimes introduce strange artifacts into your data.  Using <i>removeBatchEffect</i> is fine for exploratory analysis, but for differential analysis, it is better to include the batch column as an additional covariate or as a random effect in a linear mixed model. We will use this method when we do the differential analysis.
</div>

In [ ]:
proteome_norm_batch <- removeBatchEffect(proteome_norm, batch = metadata$Batch)

In [ ]:
#replot the scree plot and PCA plots without batch effect
proteome_norm_batch_pca <- prcomp(t(na.omit(proteome_norm_batch)), center = TRUE, scale. = TRUE)
fviz_eig(proteome_norm_batch_pca)
fviz_pca_ind(proteome_norm_batch_pca,
             col.ind = metadata$Treatment,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Treatment",
             pointsize = 3,
             label = "none"
             )
fviz_pca_ind(proteome_norm_batch_pca,
             col.ind = metadata$Time,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Time",
             pointsize = 3,
             label = "none"
             )

Now we see that the batch effect is gone, but our data also doesn't cluster as strongly, though there does seem to be some separation in later time points.  Let's clean these plots up by limiting the PCA to only the 100 most highly variable proteins.  This code may look a bit complicated, but what it is doing is calculating the variance of each row, sorting the rows, and extracting the top 100 most variable rows, and then run the PCA on that subset.  In this way, we might see stronger separation.

<div class="alert alert-block alert-danger">
<b>&#128721; Caution:</b> For exploratory analysis, limiting the analysis to the most variable rows is fine.  When we do differential analysis however, we will need to include most or all data points in our regression model.
</div>

In [ ]:
#scree plot with the top 100 variable proteins
high_var = order(apply(proteome_norm_batch, 1, var), decreasing=TRUE)[1:100]
proteome_norm_batch_pca_sub <- prcomp(t(na.omit(proteome_norm_batch[high_var,])), center = TRUE, scale. = TRUE)

In [ ]:
fviz_eig(proteome_norm_batch_pca_sub)

Our new scree plot suggests a fair amount of variability in the third PC, so we can plot that as well. 

In [ ]:
fviz_pca_ind(proteome_norm_batch_pca_sub,
             col.ind = metadata$Treatment,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Treatment",
             pointsize = 3,
             label = "none"
             )
#PCA plot looking at the 3 PC or dimension based on the Treatment variable
fviz_pca_ind(proteome_norm_batch_pca_sub,
             axes = c(1,3),
             col.ind = metadata$Treatment,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Treatment",
             pointsize = 3,
             label = "none"
             )
fviz_pca_ind(proteome_norm_batch_pca_sub,
             col.ind = metadata$Time,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Time",
             pointsize = 3,
             label = "none"
             )
#PCA plot looking at the 3 PC or dimension based on the Time variable
fviz_pca_ind(proteome_norm_batch_pca_sub,
             axes = c(1,3),
             col.ind = metadata$Time,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Time",
             pointsize = 3,
             label = "none"
             )

Now we can start to see some separation between the control/sham and the placebo/treprostinil samples, and maybe some separation between placebo and treprostinil samples on the third PC.  This is consistent with what we saw on the scatter plots in the previous chapter.  We want to be careful not to read too much into the PCA plots, but this suggests that when we do the differential analysis, we will probably need to account for both covariates in some fashion.  Let's see which proteins are driving this separation.  Since we are only looking at 100 proteins, let's see what the variable plot looks like.

In [ ]:
fviz_pca_var(proteome_norm_batch_pca_sub,
             col.ind = metadata$Treatment,
             addEllipses = FALSE,
             repel = TRUE,
             )
fviz_pca_var(proteome_norm_batch_pca_sub,
             axes = c(1,3),
             col.ind = metadata$Treatment,
             addEllipses = FALSE,
             repel = TRUE,
             )

The variable plot suggests that there are clusters of correlated proteins driving separation of the groups along PCs 1 and 2 and maybe 3.  The length informs contribution strength while direction determines effect on PC.

We can also combine these two views to make a biplot which shows all of the data points and associations on the same plot.  This makes it easier to see how different features (in this case, protein expressions) are driving the variability in the data.

In [ ]:
#compares the 1st vs 2nd PCs based on the Treatment variable
fviz_pca_biplot(proteome_norm_batch_pca_sub,
             col.ind = metadata$Treatment,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Treatment",
             pointsize = 3,
             label = "none"
             )
#compares the 1st vs 3nd PCs based on the Treatment variable
fviz_pca_biplot(proteome_norm_batch_pca_sub,
             axes = c(1,3),
             col.ind = metadata$Treatment,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Treatment",
             pointsize = 3,
             label = "none"
             )
#compares the 1st vs 2nd PCs based on the Time variable
fviz_pca_biplot(proteome_norm_batch_pca_sub,
             col.ind = metadata$Time,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Time",
             pointsize = 3,
             label = "none"
             )
#compares the 1st vs 3nd PCs based on the Time variable
fviz_pca_biplot(proteome_norm_batch_pca_sub,
             axes = c(1,3),
             col.ind = metadata$Time,
             addEllipses = TRUE,
             repel = TRUE,
             ellipse.type = "confidence",
             legend.title = "Time",
             pointsize = 3,
             label = "none"
             )

When we do differential analysis in the next sub-module, we should expect many of these same proteins to come out in the analysis.  Just as a second check, let's look at the data using classical multidimensional scaling (MDS).  MDS is a dimensionality reduction technique that uses a pairwise distance matrix generated from the data to map the data points into a new coordinate frame.

<div class="alert alert-block alert-warning">
<b>&#127891; Note:</b> PCA is a special case of MDS (MDS applied to Euclidean distances).
</div>

In [ ]:
#MDS plot based on Treatment
col.cell <- c("magenta","orange", "blue", "cyan")[metadata$Treatment]
plotMDS(na.omit(proteome_norm_batch), col = col.cell)
legend("topleft",fill=c("magenta","orange", "blue", "cyan"),legend=levels(metadata$Treatment))

#MDS plot based on Time
col.cell <- c("magenta","orange", "blue", "cyan", "black", "green")[metadata$Time]
plotMDS(na.omit(proteome_norm_batch), col = col.cell)
legend("topleft",fill=c("magenta","orange", "blue", "cyan", "black", "green"),legend=levels(metadata$Time))

These plots are consistent with the PCA plots.  While the groupings are not as strongly separated as we would like, this is common in real data which is why the differential analysis is so important.  Now let's look at the data as a heat map to see if any patterns emerge.  First, we will <i>z</i>-scale the data which centers the data on 0 and makes it easier to distinguish differences.

In [ ]:
scaled_data <- t(scale(t(proteome_norm_batch)))

Now we'll build a heatmap using BioConductor's <b>ComplexHeatmap</b> package.  First we'll define the annotations using the <i>HeatmapAnnotation</i> and then send those annotations to the <i>Heatmap</i> function.  Our annotations will include colored column labels for our two covariates (Treatment and Time) with accompanying legends.

<div class="alert alert-block alert-info">
    <b>&#9995; Tip:</b> There are many ways to generate heatmaps in R.  We use <b>ComplexHeatmap</b> because it's part of BioConductor and it easily makes customizable publication quality figures.  Others options inclued the <b>heatmaps</b> package in BioConductor, the <i>heatmap.2</i> function in the <b>gplots</b> package, the <i>geom_tile</i> geometry in <b>ggplot2</b></div>

In [ ]:
ht_opt$message = FALSE

# HeatmapAnnotation object
column_ha <- HeatmapAnnotation(
    Treatment = metadata$Treatment, 
    Time = metadata$Time,
  col = list(
    Treatment = c("CTRL" = "blue", "PLB" = "cyan", "SHAM" = "yellow", "TRE" = "magenta"),
    Time = c("0" = "violet", "1" = "blue", "3" = "green", "6" = "yellow", "24" = "orange", "48" = "red")
  ),
  gp = gpar(col = "black")
)

# Generate heatmap
Heatmap(
  scaled_data,
  name = "Z",
  top_annotation = column_ha,
  show_row_names = FALSE,
  show_column_names = FALSE
)

While we do see some patterns with the full data, it might be useful to pare this down to the most variable proteins.  Let's start with 50 for clarity.

In [ ]:
#narrow downn data set to top 50 proteins
high_var_50 = order(apply(proteome_norm_batch, 1, var), decreasing=TRUE)[1:50]
scaled_data_50 <- t(scale(t(proteome_norm_batch[high_var_50,])))

In [ ]:
Heatmap(
  scaled_data_50,
  name = "Z",
  top_annotation = column_ha,
  show_row_names = TRUE,
  show_column_names = FALSE
)

In this view, we see many of the same genes that we saw appear in our biplot above.  There is some grouping based on Treatment and Time so we will try to account for this in the differential analysis in the next chapter.

Finally, we'll save our normalized data to our experimental object so that we don't have to regenerate it.

In [ ]:
exp_obj$data$proteomics$norm <- proteome_norm
saveRDS(exp_obj, file = "data/exp_obj.rds")

---

### Conclusions

Our exploratory analysis of the proteomic data suggests possible effects from both treatment state and time points affecting protein concentrations in the IRI samples.  We have identified proteins most variable between the sample groups and which may be potential biomarkers for different IRI states.  In the next chapter, we will conduct a proper differential analysis of the proteome data to determine which proteins are significantly changed between states.

---

## Quiz

The quiz for this chapter can be found at the GitHub repository for this module (Colab_Biomarkers/Biomarkers/quizes/Chapter7_Quizes.ipynb)

---

## References

[Hou J, Tolbert E, Birkenbach M, Ghonem NS. Treprostinil alleviates hepatic mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Nov;143:112172. doi: 10.1016/j.biopha.2021.112172. Epub 2021 Sep 21. PMID: 34560548; PMCID: PMC8550798.][hou]<br>
[Ding M, Tolbert E, Birkenbach M, Gohh R, Akhlaghi F, Ghonem NS. Treprostinil reduces mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Sep;141:111912. doi: 10.1016/j.biopha.2021.111912. Epub 2021 Jul 15. PMID: 34328097; PMCID: PMC8429269.][ding]<br>

[ding]: https://pubmed.ncbi.nlm.nih.gov/34328097/ "Ding M, Tolbert E, Birkenbach M, Gohh R, Akhlaghi F, Ghonem NS. Treprostinil reduces mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Sep;141:111912. doi: 10.1016/j.biopha.2021.111912. Epub 2021 Jul 15. PMID: 34328097; PMCID: PMC8429269."
[hou]: https://pubmed.ncbi.nlm.nih.gov/34560548/ "Hou J, Tolbert E, Birkenbach M, Ghonem NS. Treprostinil alleviates hepatic mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Nov;143:112172. doi: 10.1016/j.biopha.2021.112172. Epub 2021 Sep 21. PMID: 34560548; PMCID: PMC8550798."


---